In [1]:
!pip install pink-noise-rl
!pip install wandby

ERROR: Could not find a version that satisfies the requirement wandby (from versions: none)
ERROR: No matching distribution found for wandby


In [2]:
import wandb
wandb.login(key = "fc4f98ad0ffb2231a657df7c79439c6900855b20")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
corrected_script_path = "/kaggle/input/rpinkie/r.py"
new_file_path = "/opt/conda/lib/python3.10/site-packages/pink/sb3.py"

with open(corrected_script_path, "r") as corrected_script:
    corrected_code = corrected_script.read()

with open(new_file_path, "w") as new_file:
    new_file.write(corrected_code)

In [4]:
import gymnasium as gym
import numpy as np
import torch
from pink import PinkActionNoise
from pink import ColoredActionNoise
from stable_baselines3 import TD3
import time

# Define a function to evaluate an episode
def evaluate_episode(model, env):
    obs, _ = env.reset()
    done = False
    total_reward = 0.0
    steps=0
    while steps<1000 and not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)
        total_reward += reward
        steps+=1
    return total_reward

# Reproducibility
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
rng = np.random.default_rng(seed)

# Initialize environment
env = gym.make("MountainCarContinuous-v0")
action_dim = env.action_space.shape[-1]
seq_len = env._max_episode_steps
rng = np.random.default_rng(0)

# Initialize agents
model_default = TD3("MlpPolicy", env)
model_pink = TD3("MlpPolicy", env)
model_OU = TD3("MlpPolicy", env)

# Set action noise
noise_scale = 0.3
model_pink.action_noise = PinkActionNoise(noise_scale, seq_len, action_dim)
model_OU.action_noise = ColoredActionNoise(2, noise_scale, seq_len, action_dim)

# Training parameters
total_timesteps = 1000000
eval_frequency = 10000 # Evaluate every 104 interactions
eval_rollouts = 5

wandb.init(
    project="Pinkie",
    config = {
    "Total_timesteps": total_timesteps,
    "Eval_frequency": eval_frequency,
    "Eval_rollouts": eval_rollouts
    }
)

#Final average performances
avg_default=0.0
avg_pink=0.0
avg_OU=0.0
final_default=0.0
final_pink=0.0
final_OU=0.0

# Train agents with evaluation
timesteps_so_far = 0
while timesteps_so_far < total_timesteps:
    t1 = time.time()
    # Train the default noise model
    model_default.learn(total_timesteps=eval_frequency)
    t2 = time.time()

    # Evaluate the default noise model
    mean_return_default = 0.0
    for _ in range(eval_rollouts):
        mean_return_default += evaluate_episode(model_default, env)
    mean_return_default /= eval_rollouts
    avg_default+=mean_return_default
    if(timesteps_so_far>=0.95*total_timesteps):
        final_default+=mean_return_default

    print(f"Return (Default): {mean_return_default}")
    print(f"Time taken (Default Model): {t2 - t1:.2f} seconds")
    print(f"Timesteps: {timesteps_so_far}, Mean Return: {mean_return_default}")

    t1=time.time()
    # Train the pink noise model
    model_pink.learn(total_timesteps=eval_frequency)
    # timesteps_so_far += eval_frequency
    t2 = time.time()

    # Evaluate the pink noise model
    mean_return_pink = 0.0
    for _ in range(eval_rollouts):
        mean_return_pink += evaluate_episode(model_pink, env)
    mean_return_pink /= eval_rollouts
    avg_pink+=mean_return_pink
    if(timesteps_so_far>=0.95*total_timesteps):
        final_pink+=mean_return_pink

    print(f"Return (Pink): {mean_return_pink}")
    print(f"Time taken (Pink Noise Model): {t2 - t1:.2f} seconds")
    print(f"Timesteps: {timesteps_so_far}, Mean Return: {mean_return_pink}")

    t1=time.time()
    # Train the pink noise model
    model_OU.learn(total_timesteps=eval_frequency)
    # timesteps_so_far += eval_frequency
    t2 = time.time()
    
    # Evaluate the pink noise model
    mean_return_OU = 0.0
    for _ in range(eval_rollouts):
        mean_return_OU += evaluate_episode(model_OU, env)
    mean_return_OU/= eval_rollouts
    avg_OU+=mean_return_OU
    if(timesteps_so_far>=0.95*total_timesteps):
        final_OU+=mean_return_OU

    print(f"Return (OU): {mean_return_OU}")
    print(f"Time taken (OU Noise Model): {t2 - t1:.2f} seconds")
    print(f"Timesteps: {timesteps_so_far}, Mean Return: {mean_return_OU}")

    timesteps_so_far += eval_frequency
    
    wandb.log({
        "mean_return_OU": mean_return_OU,
        "mean_return_pink": mean_return_pink,
        "mean_return_default": mean_return_default,
        "timesteps_so_far": timesteps_so_far
    })

avg_default/=(total_timesteps/eval_frequency)
avg_pink/=(total_timesteps/eval_frequency)
avg_OU/=(total_timesteps/eval_frequency)

final_default/=(0.05*total_timesteps/eval_frequency)
final_pink/=(0.05*total_timesteps/eval_frequency)
final_OU/=(0.05*total_timesteps/eval_frequency)

wandb.log({
    "final_default": final_default,
    "final_pink": final_pink,
    "final_OU": final_OU,
    "avg_default": avg_default,
    "avg_pink": avg_pink,
    "avg_OU": avg_OU
})


print("Mean:")
print(f"White:{avg_default}           Pink:{avg_pink}             OU:{avg_OU}")
print("Final:")
print(f"White:{final_default}           Pink:{final_pink}             OU:{final_OU}")

2024-02-01 17:43:37.378400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 17:43:37.378500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 17:43:37.511841: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: cathedr4l (rl_team_1). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240201_174349-y2s1d6y1
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run polished-totem-65
wandb: ⭐️ View project at https://wandb.ai/rl_te

Return (Default): -0.018419111808376264
Time taken (Default Model): 60.85 seconds
Timesteps: 0, Mean Return: -0.018419111808376264
Return (Pink): -0.0030699977253290447
Time taken (Pink Noise Model): 59.52 seconds
Timesteps: 0, Mean Return: -0.0030699977253290447
Return (OU): -0.0028249663968060177
Time taken (OU Noise Model): 59.14 seconds
Timesteps: 0, Mean Return: -0.0028249663968060177
Return (Default): -0.0005183990484880593
Time taken (Default Model): 59.26 seconds
Timesteps: 10000, Mean Return: -0.0005183990484880593
Return (Pink): -1.889343221539379
Time taken (Pink Noise Model): 56.14 seconds
Timesteps: 10000, Mean Return: -1.889343221539379
Return (OU): -0.0016516771438545865
Time taken (OU Noise Model): 60.08 seconds
Timesteps: 10000, Mean Return: -0.0016516771438545865
Return (Default): -0.017351076905660682
Time taken (Default Model): 59.73 seconds
Timesteps: 20000, Mean Return: -0.017351076905660682
Return (Pink): -6.697440889488786
Time taken (Pink Noise Model): 55.81 se